# **Connect Google Drive (files are uploaded there)**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/sapienza-camp.zip

Archive:  /content/drive/MyDrive/sapienza-camp.zip
   creating: sapienza-training-camp-2022/
  inflating: sapienza-training-camp-2022/sample_submission.csv  
   creating: sapienza-training-camp-2022/test/
  inflating: sapienza-training-camp-2022/test/out.csv  
   creating: sapienza-training-camp-2022/train/
  inflating: sapienza-training-camp-2022/train.csv  
   creating: sapienza-training-camp-2022/train/train/
   creating: sapienza-training-camp-2022/train/train/battleships/
  inflating: sapienza-training-camp-2022/train/train/battleships/00012f15.jpg  
  inflating: sapienza-training-camp-2022/train/train/battleships/004d45ed.jpg  
  inflating: sapienza-training-camp-2022/train/train/battleships/00f8dcf9.jpg  
  inflating: sapienza-training-camp-2022/train/train/battleships/01369906.jpg  
  inflating: sapienza-training-camp-2022/train/train/battleships/024dfc6c.jpg  
  inflating: sapienza-training-camp-2022/train/train/battleships/0262a137.jpg  
  inflating: sapienza-training-camp-20

# Train using pytorch tutorial for  **transfer learning**

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from sklearn.metrics import f1_score   
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.autograd import Variable
import glob
from PIL import Image
import pandas as pd

In [4]:
!rm -rf /content/sapienza-training-camp-2022/train/val/test

In [5]:
!unzip /content/drive/MyDrive/train.zip

Archive:  /content/drive/MyDrive/train.zip
   creating: train/
   creating: train/battleships/
  inflating: train/battleships/00012f15.jpg  
  inflating: train/battleships/004d45ed.jpg  
  inflating: train/battleships/00f8dcf9.jpg  
  inflating: train/battleships/01369906.jpg  
  inflating: train/battleships/024dfc6c.jpg  
  inflating: train/battleships/0262a137.jpg  
  inflating: train/battleships/0475517b.jpg  
  inflating: train/battleships/05091d90.jpg  
  inflating: train/battleships/051992ef.jpg  
  inflating: train/battleships/05a89695.jpg  
  inflating: train/battleships/05f1fa19.jpg  
  inflating: train/battleships/061124be.jpg  
  inflating: train/battleships/06e1526e.jpg  
  inflating: train/battleships/0769fa10.jpg  
  inflating: train/battleships/077770a2.jpg  
  inflating: train/battleships/07a80af4.jpg  
  inflating: train/battleships/0840bcb2.jpg  
  inflating: train/battleships/084d21eb.jpg  
  inflating: train/battleships/08ad419a.jpg  
  inflating: train/battleships/

In [5]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.RandomErasing(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/sapienza-training-camp-2022/train'

image_datasets = {'train': datasets.ImageFolder(os.path.join(data_dir, 'train'),
                                          data_transforms['train']),
                  'val':datasets.ImageFolder(os.path.join(data_dir, 'val'),
                                          data_transforms['val'])}

dataloaders = {'train':torch.utils.data.DataLoader(image_datasets['train'],
                            batch_size=4,
                            num_workers=4,
                            shuffle=True,   # You are in trainin mode, you should shuffle!
                            drop_last=True   # with this command you will have always a constant number of batch elements
                            ),
               'val':torch.utils.data.DataLoader(image_datasets['val'],
                            batch_size=4,
                            num_workers=4,
                            shuffle=False   
                            )}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [68]:
def train_model(model, criterion, optimizer, scheduler,scheduler2, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_f1 = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()            
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train' and epoch<8:
                scheduler.step()
            elif phase == 'train' and epoch>=8:
                scheduler2.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            if phase == 'val':
              meanf1score, y_true, y_pred = evalo(model, dataloaders['val'])
              print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} F1:{meanf1score*100:.4f}')

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            # deep copy the model
            if phase == 'val' and meanf1score > best_f1:
                best_f1 = meanf1score
                best_model_wts = copy.deepcopy(model.state_dict())
            
        print()
        print(f"Val meanF1score: {meanf1score*100:.4f}")
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [7]:
def evalo(model, loader):
    y_pred = []
    y_true = []
    # eval mode
    model.eval()
    for images, labels in loader:
        images = images.to(device)
        with torch.no_grad():
            logits = model(images)
        y_pred += list(torch.argmax(logits,1).cpu().numpy())
        y_true += list(labels.cpu().numpy())

    meanf1score = f1_score(y_true, y_pred, average="macro")

    return meanf1score, y_true, y_pred

In [69]:
from sklearn.metrics import f1_score
torch.manual_seed(17)

model_ft = models.resnext50_32x4d(pretrained=True)

num_ftrs = model_ft.fc.in_features
print(num_ftrs)

model_ft.fc = nn.Linear(num_ftrs, 7)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.8, nesterov=True)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=8, gamma=0.1)
exp_lr_scheduler2 = lr_scheduler.StepLR(optimizer_ft, step_size=2, gamma=0.001)


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


2048


In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,exp_lr_scheduler2,
                       num_epochs=75)
# torch.save(model_ft.state_dict(), '/content/drive/MyDrive/weights.h5')
# interrupted at 37 epochs, restarted


Epoch 0/74
----------
train Loss: 1.0348 Acc: 0.6406
val Loss: 0.2180 Acc: 0.9317 F1:89.0736
val Loss: 0.2180 Acc: 0.9317

Val meanF1score: 89.0736
Epoch 1/74
----------
train Loss: 0.7052 Acc: 0.7717
val Loss: 0.1319 Acc: 0.9545 F1:93.0652
val Loss: 0.1319 Acc: 0.9545

Val meanF1score: 93.0652
Epoch 2/74
----------
train Loss: 0.5931 Acc: 0.7961
val Loss: 0.1385 Acc: 0.9531 F1:92.7432
val Loss: 0.1385 Acc: 0.9531

Val meanF1score: 92.7432
Epoch 3/74
----------


In [ ]:
# Best val Acc: 0.975854
# 25 epochs
nesterov

resnet18 stuck 96 average f1

#resnet18 25 epochs 0.966

results on 5 epochs
sgd lr decay 7 epochs 0.1
0.93 resnet18
0.94 resnet34 (train 0.8)
0.9314 resnet50 (train 0.80)
0.9455 resnet101 (0.8277)
0.92 resnet125
0.30 convnext

# **Prediction and generating csv**

In [56]:
classes  = ['battleships', 'coast-guard', 'containerships', 'cruise-ships', 'drilling-rigs', 'motor-yachts', 'submarines']


In [57]:
model_ft.eval()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


#prediction function
def prediction(img_path,transformer1):
    
    image=Image.open(img_path)
    
    image_tensor=transformer1(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    image_tensor = image_tensor.to('cuda')
        
    input=Variable(image_tensor)
    
    
    output=model_ft(input)
    
    index=output.data.cpu().numpy().argmax()
    
    pred=classes[index]
    
    return pred
  


In [45]:
!unzip /content/drive/MyDrive/test.zip

Archive:  /content/drive/MyDrive/test.zip
   creating: test/
  inflating: test/000c110b.jpg       
  inflating: test/00268327.jpg       
  inflating: test/008fcbc9.jpg       
  inflating: test/00b9c039.jpg       
  inflating: test/00ba2385.jpg       
  inflating: test/00bf6405.jpg       
  inflating: test/00d3e184.jpg       
  inflating: test/00f6771a.jpg       
  inflating: test/00fcea78.jpg       
  inflating: test/011011c3.jpg       
  inflating: test/01242940.jpg       
  inflating: test/0125ee53.jpg       
  inflating: test/0136e092.jpg       
  inflating: test/014d8bbd.jpg       
  inflating: test/016f0d11.jpg       
  inflating: test/017b1a25.jpg       
  inflating: test/0183f117.jpg       
  inflating: test/0188cfe3.jpg       
  inflating: test/01b56869.jpg       
  inflating: test/01b80c4d.jpg       
  inflating: test/01c2565f.jpg       
  inflating: test/01d098cf.jpg       
  inflating: test/020added.jpg       
  inflating: test/020b2b5c.jpg       
  inflating: test/022fcada.

In [58]:
images_path=glob.glob('/content/test'+'/*.jpg')

In [59]:
pred_dict={}
for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,data_transforms['val'])

In [60]:
df = pd.DataFrame(pred_dict.items())
df

,0,1
0,23754976.jpg,battleships
1,441b3bb4.jpg,battleships
2,e33b3eed.jpg,battleships
3,f357feff.jpg,motor-yachts
4,1a7dca99.jpg,motor-yachts
...,...,...
2947,8e684931.jpg,motor-yachts
2948,c424fc04.jpg,drilling-rigs
2949,ae296141.jpg,battleships
2950,b45ef7f1.jpg,battleships


In [ ]:
df.groupby(1).count()

,0
1,
battleships,909
coast-guard,475
containerships,96
cruise-ships,94
drilling-rigs,725
motor-yachts,457
submarines,196


In [28]:
df.groupby(1).count()

,0
1,
battleships,913
coast-guard,468
containerships,104
cruise-ships,94
drilling-rigs,717
motor-yachts,457
submarines,199


In [61]:
a = {'motor-yachts': 5,
 'submarines': 6,
 'coast-guard': 1,
 'drilling-rigs': 4,
 'battleships': 0,
 'cruise-ships': 3,
 'containerships': 2}

for i in range(len(df)):
    df[1][i] = a[df[1][i]]
df

,0,1
0,23754976.jpg,0
1,441b3bb4.jpg,0
2,e33b3eed.jpg,0
3,f357feff.jpg,5
4,1a7dca99.jpg,5
...,...,...
2947,8e684931.jpg,5
2948,c424fc04.jpg,4
2949,ae296141.jpg,0
2950,b45ef7f1.jpg,0


In [62]:
df.columns = ['file_name','category_id']
# Saves prediction on Google Drive
df.to_csv('/content/drive/MyDrive/out.csv',index=False)